# Quick start

The following code provides a basic template for a pipeline that does model selection pipeline. Results are stored with *weight and bias*. 

In [1]:
import inferlyclient as infly
import gpflow
import numpy as np
import pandas as pd

from plotly.subplots import make_subplots
from inferlycore.models.kernels import kernel_factories, base_kernels
from inferlycore.data.transform import unit_cube_rescaling, standardise_rescaling
from inferlycore.models.diagnostics import plot_slices

file_name = '../datasets/banana.parquet'
inputs = ['x1', 'x2']
outputs = ['y']
seed = 44

use_wandb = False

anova_factory = kernel_factories['anova']
keys = ["anova" + "_" + name_kern for name_kern in base_kernels]
vals = [anova_factory(kern, len(inputs)) for kern in base_kernels.values()]
kernel_library = dict(zip(keys, vals))

if use_wandb:
    import wandb

for kern_name, kern in kernel_library.items():
    kern = kernel_library[kern_name]
    print(kern_name, kern)

    ## load data
    df = pd.read_parquet(file_name)

    dataset = infly.data.Dataset(df, inputs, outputs)
    trainset, testset = infly.data.train_test_split(dataset, n_train=1000, random_state=seed)
    testset, _ = infly.data.train_test_split(testset, n_train=500, random_state=seed)

    ## fit a GP model
    X = trainset.data[inputs]
    Y = trainset.data[outputs]

    # create transforms
    input_transform = unit_cube_rescaling(X)
    output_transform = standardise_rescaling(Y)

    # create model
    kernel = kern
    gpr = gpflow.models.GPR((input_transform.forward(X), output_transform.forward(Y)), kernel)
    gpr.likelihood.variance.assign(1e-4)
    gpmodel = infly.models.GPmodel(gpr, input_transform, output_transform)

    # optimize
    opt = gpflow.optimizers.Scipy()
    opt_logs = opt.minimize(gpmodel.model.training_loss, gpmodel.model.trainable_variables, options=dict(maxiter=200))
    print(opt_logs)


    fig = make_subplots(rows=1, cols=8, subplot_titles=inputs)
    plot_slices(gpmodel, fig, reference_point=np.zeros((1, 2)), xlim=np.array([[0, 1]]*2).T)
    #for name, param in gpflow.utilities.parameter_dict(gpr).items():
    #    wandb.log({name[1:]: param.numpy()})

    # compute metrics
    Xtest, Ytest = testset.data[inputs].values, testset.data[outputs].values
    rmse = infly.models.rmse(gpmodel, [Xtest, Ytest])
    nlpd = infly.models.nlpd(gpmodel, [Xtest, Ytest])

    print("rmse", rmse)
    print("nlpd", nlpd)

    if use_wandb:
        cfg = {
            "data_file": file_name,
            "inputs": inputs,
            "outputs": outputs,
            "kernel": kern_name,
            "seed": seed
        }

        wandb.init(
            #mode="disabled",
            project = "example-pipeline-" + '_'.join(outputs) + '_vs_' + '_'.join(inputs),
            name = kern_name + '_seed' + str(cfg['seed']),
            notes = "run generated as part of inferlycore's documentation",
            config = cfg
        )
    
        html = gpflow.utilities.tabulate_module_summary(gpmodel.model, "html")
        wandb.log({"model summary post-train": wandb.Html(html)})
        
        wandb.log({"plot prediction slices": fig})

        wandb.log({"rmse": rmse})
        wandb.log({"nlpd": nlpd})

        wandb.finish()

2023-11-18 22:28:31.929736: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-18 22:28:31.949805: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 22:28:32.035717: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-18 22:28:32.035752: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-18 22:28:32.049012: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

2023-11-18 22:28:32.797176: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ModuleNotFoundError: No module named 'inferlycore'

All done!

In [5]:
m = gpmodel.model


In [7]:
m.kernel.kernels

ListWrapper([<gpflow.kernels.base.Sum object at 0x7f40ec576680>, <gpflow.kernels.base.Sum object at 0x7f40ec575b40>])

In [9]:
m.kernel.name

'product'